<a href="https://colab.research.google.com/github/VinayakG2002/NLPlay-with-Transformers/blob/main/GPT-2%20with%20BLEU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

In [2]:
import nltk
import pandas as pd

In [4]:
url='https://raw.githubusercontent.com/VinayakG2002/NLPlay-with-Transformers/main/science.csv'
df = pd.read_csv(url)

In [5]:
for i in range (15):
  df['review'][i] = str(df['review'][i])
df['review'] = df['review'].apply(lambda x: x.replace('\n',''))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
df['length'] = df['review'].apply(len)

In [7]:

df.head(10)

,0,review,length
0,1,HE HACKER TOURIST ventures forth across the wi...,32653
1,2,Towards the end of the summer of 1969 – a few ...,23531
2,3,A Nation Of Echo Chambers: How The Internet Cl...,13626
3,4,"Decades before the rise of social media, polar...",17928
4,5,The World’s Most Efficient LanguagesHow much d...,11852
5,6,In our experience the past is the past and the...,19711
6,7,THE SOCIAL LIFE OF GENESYour DNA is not a blue...,32429
7,8,The Biology of AttractionMuch of courtship and...,26089
8,9,Extreme chemistry: experiments at the edge of ...,21271
9,10,Key trends for property investors to watch in ...,6173


In [8]:

from transformers import GPT2LMHeadModel , GPT2Tokenizer

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large') 
model = GPT2LMHeadModel.from_pretrained('gpt2-large' , 
pad_token_id = tokenizer.eos_token_id)

In [10]:
df['ref'] = df['review'].apply(lambda x: tokenizer.decode(tokenizer.encode(x[:2000],return_tensors='pt')[0]))

In [ ]:
def gen(x):
  input_ids = tokenizer.encode(x[:200],return_tensors='pt')
  output = model.generate(input_ids, max_length = 2000, num_beams = 5,no_repeat_ngram_size  = 2,early_stopping = True)
  return output

df['gen'] = df['ref'].apply(gen)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu


def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []
    for l in gen:
        gen_bleu.append(l.split())
    for i,l in enumerate(ref):
        ref_bleu.append([l.split()])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu

In [ ]:
x = bleu(list(df['ref']),list(df['gen'])